In [1]:
from funcs.notebook import *

from astropy.constants import R_sun, L_sun
import astropy.units as u

In [2]:
# BT Settl model
bt = pd.read_csv("../data/BTSETTL_AUMic_lte037-5.0-0.0a+0.0.BT-NextGen.7.dat.txt",
                skiprows=8, names=["lambd",'flux'], sep="\s+")

# TESS bandpass
tess = pd.read_csv("../data/TESS_response.txt", skiprows=8,sep="\s+")

In [3]:
# convert nm to Angstrom
tess["lambd"] = tess.nm * 10.
tess.head()

,nm,response,lambd
0,500.035217,0.004863,5000.35217
1,533.764648,0.004863,5337.64648
2,565.076416,0.007879,5650.76416
3,580.145508,0.007879,5801.45508
4,600.501709,0.826957,6005.01709


In [4]:
# Interpolate response from rwav to w:
rres = np.interp(tess.lambd, bt.lambd, bt.flux, left=0, right=0)

# Integrating the flux of the thermal
# spectrum times the response curve over wavelength:
tessflux = np.trapz(tess.response * rres, x=tess.lambd)*u.erg * u.cm**(-2) * u.s**(-1)

# literature values from White et al. 2015
r, rerr = 0.75 * R_sun, 0.03 * R_sun 

# TESS projected surface luminosity
tesslum = np.pi * r**2 * tessflux

# convert to astro units
tesslum = tesslum.to("erg /s")
tesslum

<Quantity 2.32979173e+31 erg / s>

In [5]:
with open("../results/values/tesslum_aumic.txt", "w") as f:
    f.write(str(tesslum.value))

In [6]:
# compare with bolomteric flux
(np.trapz(bt.flux , x=bt.lambd)*u.erg * u.cm**(-2) * u.s**(-1) * 4. * np.pi * r**2).to("erg/s")

<Quantity 3.56817621e+32 erg / s>